In [1]:
#!pip install pybaseball

In [2]:
from pybaseball import pitching_stats
import pandas as pd
import numpy as np

In [3]:
#Brings in both seasonal and career pitching statistics for players in the MLB from 2007 until 2022
career_pitching = pitching_stats(2007, 2022, qual=10, ind=0)
seasonal_pitching = pitching_stats(2007, 2022, qual=10, ind=1)

In [4]:
career_pitching.reset_index(inplace=True)
seasonal_pitching.reset_index(inplace=True)

### Test Stuff

In [5]:
seasonal_pitching.head()

,index,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,...,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA
0,270,10954,2018,Jacob deGrom,NYM,30,10,9,9.0,1.70,...,11.1,20.0,0.039,112.9,148.0,0.287,515,0.164,0.315,NaN
1,798,1303,2011,Roy Halladay,PHI,34,19,6,8.7,2.35,...,NaN,0.0,NaN,NaN,0.0,NaN,0,0.185,0.294,NaN
2,582,1943,2009,Zack Greinke,KCR,25,16,8,8.7,2.16,...,NaN,0.0,NaN,NaN,0.0,NaN,0,0.187,0.287,NaN
3,549,2036,2015,Clayton Kershaw,LAD,27,16,7,8.6,2.13,...,8.9,15.0,0.028,113.3,132.0,0.244,542,0.164,0.323,NaN
4,2832,8700,2009,Justin Verlander,DET,26,19,9,8.4,3.45,...,NaN,0.0,NaN,NaN,0.0,NaN,0,0.175,0.289,NaN


In [6]:
#Data Engineering - Creating a total seasons column
tot_seasons = []
for i in range(0, len(career_pitching)):
    count = 0
    
    for j in range(0, len(seasonal_pitching)):
        if seasonal_pitching['Name'][j] == career_pitching['Name'][i]:
            count += 1
    tot_seasons.append(count)
            

In [7]:
career_pitching['Seasons'] = tot_seasons

In [8]:
career_pitching.head()

,index,IDfg,Name,Team,Age,W,L,ERA,WAR,G,...,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA,Seasons
0,244,8700,Justin Verlander,- - -,31,227,122,3.20,75.1,450,...,178.0,0.060,115.9,977.0,0.328,2981,0.169,0.280,NaN,14
1,52,2036,Clayton Kershaw,LAD,27,197,87,2.48,73.4,401,...,156.0,0.051,115.9,943.0,0.305,3087,0.173,0.297,NaN,15
2,191,3137,Max Scherzer,- - -,30,201,102,3.11,70.3,430,...,222.0,0.064,114.7,1096.0,0.318,3446,0.169,0.303,NaN,15
3,261,1943,Zack Greinke,- - -,31,209,113,3.24,61.7,496,...,244.0,0.061,117.4,1333.0,0.331,4028,0.180,0.281,NaN,16
4,328,4972,Cole Hamels,- - -,30,154,114,3.39,49.1,400,...,147.0,0.057,118.3,922.0,0.361,2557,0.162,0.283,NaN,13


In [9]:
#Saving the names of every player who played at least 10 seasons
count = 0
names_for_forecasting = []
for i in range(0, len(career_pitching)):
    if career_pitching['Seasons'][i] > 9:
        names_for_forecasting.append(career_pitching['Name'][i])
        count += 1

In [10]:
len(names_for_forecasting)

191

In [11]:
#Now interested in creating dataframes(For Time Series) of every player who played at least 10 seasons

In [12]:
#Creating a list that holds a dataframe of season by season data, for every player in our names_for_forecasting list

#List to hold our dataframes
forecasting_dfs = []

#curr_row = {'Date': seasonal_pitching['Season'][j], 'Name': seasonal_pithching['Name'][j], 'Age': seasonal_pithching['Age'][j], 'W': seasonal_pithching['W'][j], 'L': seasonal_pithching['L'][j], 'G': seasonal_pithching['G'][j], 'WAR': seasonal_pithching['WAR'][j], 'ERA': seasonal_pithching['ERA'][j], 'K/9': seasonal_pithching['K/9'][j], 'BB': seasonal_pithching['BB'][j]}    


for i in range(0, len(names_for_forecasting)):
    
    df = pd.DataFrame()
    curr_name = names_for_forecasting[i]
    
    for j in range(0, len(seasonal_pitching)):
        if seasonal_pitching['Name'][j] == curr_name:
            curr_row = {'Date': seasonal_pitching['Season'][j], 'Name': seasonal_pitching['Name'][j], 'Age': seasonal_pitching['Age'][j], 'W': seasonal_pitching['W'][j], 'L': seasonal_pitching['L'][j], 'G': seasonal_pitching['G'][j], 'WAR': seasonal_pitching['WAR'][j], 'ERA': seasonal_pitching['ERA'][j], 'K/9': seasonal_pitching['K/9'][j], 'BB': seasonal_pitching['BB'][j], 'BABIP': seasonal_pitching['BABIP'][j]} 
            df = df.append(curr_row, ignore_index=True)
    forecasting_dfs.append(df)


/var/folders/pr/ffd0v18136d92642k7xbzgb80000gn/T/ipykernel_31677/272488900.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(curr_row, ignore_index=True)
/var/folders/pr/ffd0v18136d92642k7xbzgb80000gn/T/ipykernel_31677/272488900.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(curr_row, ignore_index=True)
/var/folders/pr/ffd0v18136d92642k7xbzgb80000gn/T/ipykernel_31677/272488900.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(curr_row, ignore_index=True)
/var/folders/pr/ffd0v18136d92642k7xbzgb80000gn/T/ipykernel_31677/272488900.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  

In [13]:
#Veryifying code worked as intended, it did
forecasting_dfs[1]

,Date,Name,Age,W,L,G,WAR,ERA,K/9,BB,BABIP
0,2015,Clayton Kershaw,27,16,7,33,8.6,2.13,11.64,42,0.281
1,2014,Clayton Kershaw,26,21,3,27,7.9,1.77,10.85,31,0.278
2,2011,Clayton Kershaw,23,21,5,33,7.3,2.28,9.57,54,0.269
3,2013,Clayton Kershaw,25,16,9,33,7.2,1.83,8.85,52,0.251
4,2016,Clayton Kershaw,28,12,4,21,6.3,1.69,10.39,11,0.254
5,2012,Clayton Kershaw,24,14,9,33,6.1,2.53,9.05,63,0.262
6,2010,Clayton Kershaw,22,13,10,32,4.6,2.91,9.34,81,0.275
7,2017,Clayton Kershaw,29,18,4,27,4.3,2.31,10.39,30,0.267
8,2009,Clayton Kershaw,21,8,8,31,4.1,2.79,9.74,91,0.269
9,2022,Clayton Kershaw,34,12,3,22,3.8,2.28,9.76,23,0.268


In [14]:
#Want to filter our list to only a pitcher who did pitch last season
pitched_2022 = []
for i in range(0, len(forecasting_dfs)):
    
    if forecasting_dfs[i]['Date'].max() == 2022:
        pitched_2022.append(forecasting_dfs[i])
    

In [49]:
#For loop just to get the names of each pitcher
just_names = []
for i in range(0, len(pitched_2022)):
    just_names.append(pitched_2022[i]['Name'][2022])

In [50]:
tuple(just_names)

('Justin Verlander',
 'Clayton Kershaw',
 'Max Scherzer',
 'Zack Greinke',
 'Adam Wainwright',
 'David Price',
 'Corey Kluber',
 'Gerrit Cole',
 'Madison Bumgarner',
 'Jose Quintana',
 'Johnny Cueto',
 'Yu Darvish',
 'Lance Lynn',
 'Anibal Sanchez',
 'Carlos Carrasco',
 'Charlie Morton',
 'Kevin Gausman',
 'Sonny Gray',
 'Patrick Corbin',
 'Kenley Jansen',
 'Chris Archer',
 'Dallas Keuchel',
 'Craig Kimbrel',
 'Aroldis Chapman',
 'Nathan Eovaldi',
 'Wade Miley',
 'Rich Hill',
 'Kyle Gibson',
 'Alex Wood',
 'Mike Minor',
 'Alex Cobb',
 'Jhoulys Chacin',
 'Jake Odorizzi',
 'Martin Perez',
 'Collin McHugh',
 'Ian Kennedy',
 'David Robertson',
 'Liam Hendriks',
 'Michael Wacha',
 'Garrett Richards',
 'Mark Melancon',
 'Daniel Hudson',
 'Sergio Romo',
 'Ryan Pressly',
 'Matt Moore',
 'Steve Cishek',
 "Darren O'Day",
 'Adam Ottavino',
 'Craig Stammen',
 'Jake McGee',
 'Brad Hand',
 'Tommy Hunter',
 'Jordan Lyles',
 'Jesse Chavez',
 'Tommy Milone',
 'Joe Smith',
 'David Phelps',
 'Will Smith'

In [15]:
#Now, want to get all 74 usable dataframes into time series formatting
for i in range(0, len(pitched_2022)):
    pitched_2022[i].set_index('Date', inplace = True)
    pitched_2022[i].sort_index(inplace=True)
    

In [25]:
#Data is ready for time series analysis
pitched_2022[0]

,Name,Age,W,L,G,WAR,ERA,K/9,BB,BABIP
Date,,,,,,,,,,
2007,Justin Verlander,24,18,6,32,3.7,3.66,8.17,67,0.279
2008,Justin Verlander,25,11,17,33,3.2,4.84,7.30,87,0.296
2009,Justin Verlander,26,19,9,35,8.4,3.45,10.09,63,0.319
2010,Justin Verlander,27,18,9,33,6.7,3.37,8.79,71,0.286
2011,Justin Verlander,28,24,5,34,6.5,2.40,8.96,57,0.236
2012,Justin Verlander,29,17,8,33,6.9,2.64,9.03,60,0.273
2013,Justin Verlander,30,13,12,34,4.8,3.46,8.95,75,0.316
2014,Justin Verlander,31,15,12,32,3.2,4.54,6.95,65,0.317
2015,Justin Verlander,32,5,8,20,3.1,3.37,7.63,32,0.267


In [26]:
#Learned about storing variables from https://stackoverflow.com/questions/31621414/share-data-between-ipython-notebooks
%store pitched_2022

Stored 'pitched_2022' (list)


In [34]:
career_pitching.isnull().sum().sort_values(ascending=False)

xERA          2459
UN% (sc)      2459
vSC (sc)      2455
SC% (sc)      2455
wSC/C (sc)    2455
              ... 
Swing%           0
O-Contact%       0
Z-Contact%       0
Contact%         0
Seasons          0
Length: 335, dtype: int64

In [35]:
x = career_pitching.columns

In [38]:
no_nulls = []
for i in range(0, 335):
    if career_pitching[x[i]].isnull().sum() == 0:
        no_nulls.append(x[i])

In [42]:
for i in range(0, len(no_nulls)):
    print(no_nulls[i])

index
IDfg
Name
Team
Age
W
L
ERA
WAR
G
GS
CG
ShO
SV
BS
IP
TBF
H
R
ER
HR
BB
IBB
HBP
WP
BK
SO
GB
FB
LD
IFFB
Balls
Strikes
Pitches
RS
IFH
BU
BUH
K/9
BB/9
K/BB
H/9
HR/9
AVG
WHIP
BABIP
LOB%
FIP
GB/FB
LD%
GB%
FB%
IFFB%
HR/FB
IFH%
BUH%
RAR
Dollars
tERA
xFIP
WPA
-WPA
+WPA
RE24
REW
pLI
inLI
gmLI
Pulls
WPA/LI
Clutch
O-Swing%
Z-Swing%
Swing%
O-Contact%
Z-Contact%
Contact%
Zone%
F-Strike%
SwStr%
HLD
SD
MD
ERA-
FIP-
xFIP-
K%
BB%
SIERA
RS/9
E-F
RA9-WAR
BIP-Wins
LOB-Wins
FDP-Wins
Age Rng
K-BB%
Pull%
Cent%
Oppo%
Soft%
Med%
Hard%
kwERA
TTO%
K/9+
BB/9+
H/9+
HR/9+
AVG+
WHIP+
BABIP+
LOB%+
K%+
BB%+
LD%+
GB%+
FB%+
HR/FB%+
Pull%+
Cent%+
Oppo%+
Soft%+
Med%+
Hard%+
Events
CStr%
CSW%
Seasons


In [43]:
#Making Stat Specific Dataframes
#Career Data - Key is to only include rate statisics, since not all pitches have the same workload in their respective roles
df_career = pd.DataFrame()
df_career['Seasons'] = career_pitching['Seasons']
df_career['H'] = career_pitching['H']
df_career['WAR'] = career_pitching['WAR']
df_career['ERA'] = career_pitching['ERA']
df_career['K/9'] = career_pitching['K/9']
df_career['H/9'] = career_pitching['H/9']
df_career['ER'] = career_pitching['ER']
df_career['HBP'] = career_pitching['HBP']
df_career['K-BB%'] = career_pitching['K-BB%']
df_career['Contact%'] = career_pitching['Contact%']
df_career['FB%'] = career_pitching['FB%']

### Exporting Career MLB Player Data (From 2007)

In [51]:
df_career.to_csv('../Data/career_pitcher.csv', index=False)